# Azure OpenAI Service Completion 기초
Azure OpenAI Service에는 다양한 업무에 사용가능한 입력 완성(Completion) 엔드포인트가 있다. 이 엔드포인트는 단순하면서도 강력한 '텍스트 입력-텍스트 출력' 인터페이스를 모든 Azure OpenAI 모델에 제공한다. 입력한 문장을 완성시키려면 **프롬프트**를 전달해야 한다. 모델은 입력한 텍스트를 완성시킬 후보군을 생성하여 문맥이나 패턴에 따라 문장을 완성한다. 가령 '데카르트의 유명한 말인 나는 생각한다 그러므로'라는 프롬프트를 API에 전달하면 Azure OpenAI는 이 프롬프트에 기반하여 높은 확률을 가진 후보군 중에서 '나는 존재한다'를 선택해서 반환할 것이다.

[Azure OpenAI Studio](https://oai.azure.com/)의 플레이그라운드를 사용하면 텍스트를 완성시킬 후보군을 확인할 수 있다.

## 사전 준비

이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:

- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service에 배포된 GPT-3.5 Turbo 모델. 이 예제에서 사용하는 API 버전은 `2024-02-01`다. 배포 이름은 모델과 같은 `gpt-35-turbo-instruct`를 사용한다.
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI GPT-3.5 Turbo Instruct 모델의 배포 이름
  - OpenAI API 버전
- Python(이 예제는 버전 3.12.4로 테스트 했다.)

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.


## 패키지 설치


In [ ]:
!pip install openai

In [ ]:
import openai
openai.__version__

## 라이브러리 및 환경변수 불러오기


## Azure OpenAI 설정
Azure OpenAI와 연동을 위해 필요한 정보는 보안을 위해 하드코딩 하지 말고 환경변수나 [dotenv](https://pypi.org/project/python-dotenv/)로 불러오는 것을 권장한다.

In [ ]:
import os

#from dotenv import load_dotenv
#load_dotenv()

os.environ["AZURE_OPENAI_API_KEY"] = "Your OpenAI API Key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://<Your OpenAI Service>.openai.azure.com/"

# 이 변수에는 모델을 배포했을 때 설정한 커스텀 이름을 입력한다. 이 예제에서는 gpt-35-turbo-instruct를 사용했다.
AZURE_OPENAI_INSTRUCT_DEPLOYMENT = "gpt-35-turbo-instruct"

## Hello world!

### 텍스트 생성 - 아이디어 제안
모델은 새로운 아이디어나 여러 버전의 결과물이 필요할 때 자주 사용된다. 가령 미스테리 소설을 집필하는 데 아이디어가 필요한 경우 모델에 아이디어가 필요한 스토리를 제공하면 모델은 제안받은 스토리에 기반하여 사업 계획, 캐릭터 설정, 마케팅 슬로건 등 아이디어를 추가해준다.  

https://learn.microsoft.com/azure/ai-services/openai/how-to/completions


In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

In [ ]:
# Send a completion call to generate an answer
prompt = "아이스크림 가게의 캐치프레이즈를 만들어줘."
response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT, prompt=prompt, max_tokens=100
)
print(prompt)
print(response.choices[0].text)

위 예시는 문맥이 거의 없어서 모델이 기대한 결과를 반환하지 않을 수 있다. 응답이 예상과 다르거나 잘린 것처럼 보이면 토큰의 최대수(`max_tokens`)를 조정하면서 실행해보자.

## 코드 완성하기
다음과 같은 React 컴포넌트의 코드를 완성 기능으로 보완할 수 있다. 예시 코드를 보면 모델에 코드를 전달하되 여는 괄호(`(`)까지만 코드를 입력한 것을 볼 수 있다. 이렇게 하면 모델은 여는 괄호까지만 작성된 HeaderComponent의 불완전한 코드를 보완할 트리거로 해석한다. 모델은 코드 작성에 필요한 React 라이브러리 지식도 있기 때문에 코드를 완성시킬 수 있다.

https://learn.microsoft.com/azure/ai-services/openai/how-to/completions#complete-partial-text-and-code-inputs

In [ ]:
# Generation prompt
prompt = """
import React from 'react';
const HeaderComponent = () => (
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

## 텍스트 요약
모델은 텍스트의 맥락을 파악하고 이를 다양한 방법으로 변환할 수 있다. 이 예제에서 모델은 텍스트 블록을 받아 초등학생도 이해할 수 있게 설명을 생성한다. 이는 모델이 언어를 깊이 이해하고 있음을 보여준다.

https://learn.microsoft.com/azure/ai-services/openai/how-to/completions#summarize-text

In [ ]:
# Generation prompt
prompt = """
다음 설명을 100자 이내로 요약해줘.

###
중성자별은 초신성 폭발 직후 무거운 별이 중력붕괴하여 만들어진 밀집성의 일종이다.
중성자별은 현재까지 관측된 우주의 천체 중 블랙홀 다음으로 밀도가 크다. 거의 12 ~ 13 km의 반지름에 태양의 두 배에 달하는 무거운 질량을 가지고 있다.
중성자별은 거의 대부분이 순전하가 없고 양성자보다 약간 더 무거운 핵자인 중성자로 구성되어 있다. 
이들은 양자 축퇴압에 의해 붕괴되지 않고 유지되는데 이는 매우 뜨거우며 두 개의 중성자(또는 페르미 입자)가 동시에 같은 위치 및 양자 상태를 취할 수 없다는 원리인 파울리 배타 원리를 통해 설명되는 현상이다.
중성자별의 질량은 최소 1.1 태양질량에서 3 태양질량(M☉)까지이다. 관측된 것 중 가장 무거운 것은 2.01 M☉이다. 
중성자별의 표면온도는 보통 ~6×105 K이다. 중성자별의 전체 밀도는 3.7×1017에서 5.9×1017 kg/m3 (태양의 밀도의 2.6×1014 ~ 4.1×1014 배)이다.
###

"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

In [ ]:
# Generation prompt
prompt = """
다음 설명을 6살인 유치원생도 이해할 수 있게 요약해줘.
유치원생이 모르는 단어는 쉬운 단어로 변환해줘.

###
중성자별은 초신성 폭발 직후 무거운 별이 중력붕괴하여 만들어진 밀집성의 일종이다.
중성자별은 현재까지 관측된 우주의 천체 중 블랙홀 다음으로 밀도가 크다. 거의 12 ~ 13 km의 반지름에 태양의 두 배에 달하는 무거운 질량을 가지고 있다.
중성자별은 거의 대부분이 순전하가 없고 양성자보다 약간 더 무거운 핵자인 중성자로 구성되어 있다. 
이들은 양자 축퇴압에 의해 붕괴되지 않고 유지되는데 이는 매우 뜨거우며 두 개의 중성자(또는 페르미 입자)가 동시에 같은 위치 및 양자 상태를 취할 수 없다는 원리인 파울리 배타 원리를 통해 설명되는 현상이다.
중성자별의 질량은 최소 1.1 태양질량에서 3 태양질량(M☉)까지이다. 관측된 것 중 가장 무거운 것은 2.01 M☉이다. 
중성자별의 표면온도는 보통 ~6×105 K이다. 중성자별의 전체 밀도는 3.7×1017에서 5.9×1017 kg/m3 (태양의 밀도의 2.6×1014 ~ 4.1×1014 배)이다.
###

"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

Azure OpenAI API는 매번 새로운 출력을 생성하기 때문에 표시되는 결과가 다를 수 있다. 이는 동일한 프롬프트를 사용해도 마찬가지다. 만약 동일한 답변을 받고 싶다면 `temperature`를 조정하면 된다.

## 텍스트 분류
텍스트 분류기를 만들려면 문제를 설명하면서 몇 가지 예시를 제공해야 한다. 아래 예시에서는 문자 메시지의 감정을 분류하는 방법을 모델에 제공하고 있다. 여기서 감정이란 텍스트의 전체적인 감정 혹은 특정 표현에 대한 감정을 의미한다.

https://learn.microsoft.com/azure/ai-services/openai/how-to/completions

In [ ]:
prompt = """
너는 문자 메시지의 감정을 분류하는 분류기야. 

Message: "이번에 개봉한 어드벤처 영화인데 완전 재밌어요!"
Sentiment: 긍정

Message: "핸드폰 배터리가 방전됐어. 짜증나." 
Sentiment: 부정

Message: "오늘 날씨가 👍"
Sentiment: 긍정

Message: "말씀드렸던 기사 링크입니다."
Sentiment: 중립

Message: "신작 뮤비가 비현실적으로 멋있어."
Sentiment: 
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

### API 요청 1회로 여러 결과 획득하기
분류기를 만드는 방법을 이해했다면 이번에는 위에서 사용한 예제를 확장해서 효율성을 높여보자. 다음과 같이 API를 1회 요청하여 여러 결과를 획득할 수 있다.

In [ ]:
prompt = """
너는 문자 메시지의 감정을 분류하는 분류기야. 

Message: "이번에 개봉한 어드벤처 영화인데 완전 재밌어요!"
Sentiment: 긍정

Message: "핸드폰 배터리가 방전됐어. 짜증나." 
Sentiment: 부정

Message: "오늘 날씨가 👍"
Sentiment: 긍정

Message: "말씀드렸던 기사 링크입니다."
Sentiment: 중립

문자 메시지
1. "이번에 개봉한 어드벤처 영화인데 완전 재밌어요!"
2. "핸드폰 배터리가 방전됐어. 짜증나."
3. "오늘 날씨가 👍"
4. "말씀드렸던 기사 링크입니다."
5. "신작 뮤비가 비현실적으로 멋있어."

메시지 감정 평가
1: 긍정
2: 부정
3: 긍정
4: 중립
5: 긍정

문자 메시지
1. "걔는 숙제를 진짜 싫어해."
2. "택시가 늦게와서 여자친구가 화났어😠"
3. "주말이 기대되네."
4. "우리집 고양이 진짜 귀엽지 않냐?❤️❤️"
5. "초코 바나나 먹어봐"

메시지 감정 평가
1:
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

## 텍스트 번역
한국어 문장을 불어로 번역하는 방법을 모델에 지시해보자.

In [ ]:
prompt = """
Korean: 나는 불어는 못해.
French: Je ne parle pas français.
Korean: 또 만나자!
French: À tout à l'heure!
Korean: 추천하는 식당 있어?
French: Où est un bon restaurant?
Korean: 빈 방 있나요?
French: Quelles chambres avez-vous de disponible?
Korean: 가장 가까운 역이 어디에요?
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

### 이모지 변환
영화 제목을 텍스트에서 이모지로 변환한다. 이 예시는 모델이 패턴을 파악해서 다른 문자를 활용하는 적응력이 있음을 보여준다.

In [ ]:
prompt = """
Carpool Time: 👨👴👩🚗🕒
Robots in Cars: 🚗🤖
Super Femme: 👸🏻👸🏼👸🏽👸🏾👸🏿
Webs of the Spider: 🕸🕷🕸🕸🕷🕸
The Three Bears: 🐻🐼🐻
Mobster Family: 👨👩👧🕵🏻‍♂️👲💥
Arrows and Swords: 🏹🗡🗡🏹
Snowmobiles:
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
    top_p=1.0,
)
print(response.choices[0].text)

## 질의응답(Q&A)
모델은 기업 내부 문서 등 학습한 적 없는 다양한 문서의 문맥을 파악해서 답변할 수 있다.

In [ ]:
prompt = """
다음 텍스트를 사용해서 아래 질문에 답변해줘. 
답변할 수 없는 질문에는 '모르겠습니다'라고만 답변해줘.

문맥:
###
서피스북의 배터리는 0%에서 100%까지 충전하는 데 2~4시간이 소요됩니다.
서피스북을 충전하는 동안 게임이나 동영상 스트리밍 등 전력 소비가 많은 작업을 하면 더 많은 시간이 소요될 수 있습니다.
충전 어댑터에 연결된 USB 포트를 사용해서 서비스북 충전중에 스마트폰 등 다른 디바이스를 충전할 수도 있습니다.
충전 어댑터의 USB 포트는 충전전용이라 데이터 전송은 불가능합니다.
###

질문: 서피스북의 충전시간을 줄이는 방법을 알려줘.
답변: 
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.0,
    top_p=0.9,
)
print(response.choices[0].text)

### 템플릿화
프롬프트를 템플릿화해서 프로그램과 통합하거나 편하게 관리할 수 있다.

In [ ]:
prompt_template = """
다음 텍스트를 사용해서 아래 질문에 답변해줘.
답변할 수 없는 질문에는 '모르겠습니다'라고만 답변해줘.

문맥: 
###
{input}
###

질문: {question}
답변: 
"""

prompt = prompt_template.format(
    input="서피스북의 배터리는 0%에서 100%까지 충전하는 데 2~4시간이 소요됩니다. 서피스북을 충전하는 동안 게임이나 동영상 스트리밍 등 전력 소비가 많은 작업을 하면 더 많은 시간이 소요될 수 있습니다. 충전 어댑터에 연결된 USB 포트를 사용해서 서비스북 충전중에 스마트폰 등 다른 디바이스를 충전할 수도 있습니다. 충전 어댑터의 USB 포트는 충전전용이라 데이터 전송은 불가능합니다.",
    question="서피스북의 충전시간을 줄이는 방법을 알려줘.",
)
print(prompt)

In [ ]:
response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.0,
    top_p=0.9,
)
print(response.choices[0].text)

## 엔티티 추출
다음은 텍스트에서 엔티티(이름, 지명, 조직 등)를 추출하는 예제다.

In [ ]:
prompt = """
아래 이메일 내용에서 발신자의 이름과 주소를 추출해줘.

###
승민님! 세미나에서 같이 얘기할 수 있어서 정말 영광이었어요.
여진님 발표도 진짜 도움 많이 됐어요.
도서 이벤트까지 해주셔서 정말 감사해요.
제 주소는 경북 울릉군 울릉읍 독도안용복길 3이에요.

이순신 드림
###
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.1,
    top_p=0.9,
)
print(response.choices[0].text)

## Zero-shot CoT(생각의 사슬)
생각의 사슬은 예시 없이 문제 해결을 유도하는 방법("차근차근 생각해 봐(Let's think step by step)")이다. 복잡한 질문이나 지시를 작은 문제들로 분해해서 문제별로 해결 방안을 마련하는 매우 중요한 기법이다.


In [ ]:
prompt = """
질문: 존한테 5개의 사과가 있었는데 2개를 먹고 5개를 더 산 다음 3개를 친구한테 줬다면 존이 가지고 있는 사과는 몇 개일까?
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.1,
    top_p=0.9,
)
print(response.choices[0].text)

In [ ]:
prompt = """
질문: 존한테 5개의 사과가 있었는데 2개를 먹고 5개를 더 산 다음 3개를 친구한테 줬다면 존이 가지고 있는 사과는 몇 개일까?

추론과정을 확인할 수 있게 단계적으로 기술해줘.
"""

response = client.completions.create(
    model=AZURE_OPENAI_INSTRUCT_DEPLOYMENT,
    prompt=prompt,
    max_tokens=200,
    temperature=0.1,
    top_p=0.9,
)
print(response.choices[0].text)